In [1]:
import subprocess
from functools import partial
import logging
from pathlib import Path
from time import strftime, gmtime
from typing import List

import numpy as np
import pandas as pd
from tqdm import tqdm

from ilids.utils.ffmpeg_tqdm import analyze_stream, ffmpeg_to_tqdm

In [2]:
def get_ffmpeg_select_argument(frames_sequence: List[int], N: int) -> str:
    """Produce a string to be used with the FFMPEG command and its "select" argument.

    :param frames_sequence: linear or not list of frames to extract.
                            Example: 0, 12, 24, 36, 48, 60, ...
                            or:      6, 23, 27, 42, 50, 68, ...
    :param setpts_filter: from FFMPEG manual page:
                            "setpts" filter, which only sets timestamps and otherwise passes the frames unchanged
                          It is expected to be smaller than 1 in can less frames are selected and it is wished to
                          speed up the video.
                          Usually, it is expected to get 1/FRAMES_SKIP
                          Example: *0.25* (for a FRAMES_SKIP of 4)
    :return: string to be used to run the FFMPEG command for the "select" argument like: ffmpeg -i SZTRA201a08_224.mp4 -vf select='eq(n\,0)+eq(n\,12)+eq(n\,24)+eq(n\,36)+eq(n\,48)+eq(n\,60), setpts=0.08333333333*PTS' -an SZTRA201a08_224_12_random.mp4
    """
    # eq(n\\,0)+eq(n\\,12)+eq(n\\,24)+eq(n\\,36)+eq(n\\,48)
    eq_chain = "+".join([f"eq(n\,{n})" for n in frames_sequence])
    # change the 'setpts' filter to correct the shift in frames in case of any.
    # Else, starting timestamp will be "FIRST FRAME" / N (and you will miss most of the desired sequence)
    setpts = f"setpts={1 / N}*PTS-{int(frames_sequence[0] / N)}"
    return f"'{eq_chain}',{setpts}"


def trivial(interval: int, frames_count: int):
    return list(range(0, frames_count, interval)) + (
        [] if (frames_count - 1) % interval == 0 else [frames_count - 1]
    )

In [3]:
# # Adapted from CLIP4Clip: https://github.com/ArrowLuo/CLIP4Clip/blob/master/preprocess/compress_video.py
# # but don't change the frame rate as be which to have different frame selection strategies
# # and use ffmpeg_to_tqdm helper
# def scale_compress_video(input_video_path: Path, output_video_path: Path) -> int:
#     logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
#     logger.debug("Using ffmpeg to scale and compress an original video.")

#     stream_info = analyze_stream(logger, str(input_video_path))

#     command = [
#         "ffmpeg",
#         "-y",  # (optional) overwrite output file if it exists
#         "-i",
#         str(input_video_path),
#         "-filter:v",
#         "scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)'",  # scale to 224
#         "-map",
#         "0:v",
#         "-vcodec",
#         "libx264",  # additionally, change the codec for better
#         # compression
#         # display codecs with: `ffmpeg -codecs`
#         #    '-r', '3',  # frames per second
#         str(output_video_path),
#     ]
#     # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
#     # progress, use 'universal_newlines' argument
#     # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
#     ffmpeg_process = subprocess.Popen(
#         command,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.STDOUT,
#         universal_newlines=True,
#     )

#     return ffmpeg_to_tqdm(
#         logger,
#         ffmpeg_process,
#         duration=stream_info.get("duration"),
#         tqdm_desc="FFMPEG scale down to 224 and encode libx264",
#     ).returncode

# # Using Stackoverflow answer, we can select multiple frames with the '+' "operation" inside the select
# def get_ffmpeg_select_argument(frames_sequence: List[int], setpts_filter: float) -> str:
#     """Produce a string to be used with the FFMPEG command and its "select" argument.

#     :param frames_sequence: linear or not list of frames to extract.
#                             Example: 0, 12, 24, 36, 48, 60, ...
#                             or:      6, 23, 27, 42, 50, 68, ...
#     :param setpts_filter: from FFMPEG manual page:
#                             "setpts" filter, which only sets timestamps and otherwise passes the frames unchanged
#                           It is expected to be smaller than 1 in can less frames are selected and it is wished to
#                           speed up the video.
#                           Usually, it is expected to get 1/FRAMES_SKIP
#                           Example: *0.25* (for a FRAMES_SKIP of 4)
#     :return: string to be used to run the FFMPEG command for the "select" argument like: ffmpeg -i SZTRA201a08_224.mp4 -vf select='eq(n\,0)+eq(n\,12)+eq(n\,24)+eq(n\,36)+eq(n\,48)+eq(n\,60), setpts=0.08333333333*PTS' -an SZTRA201a08_224_12_random.mp4
#     """
#     # eq(n\\,0)+eq(n\\,12)+eq(n\\,24)+eq(n\\,36)+eq(n\\,48)
#     eq_chain = "+".join([f"eq(n\,{n})" for n in frames_sequence])
#     setpts = f"setpts={setpts_filter}*PTS"
#     return f"'{eq_chain}',{setpts}"
#
# def extract_frames_in_video(
#     frames_sequence: List[int],
#     setpts_filter: float,
#     input_video_path: Path,
#     output_video_path: Path,
# ) -> int:
#     logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
#     logger.setLevel(logging.INFO)
#     logger.debug("Using ffmpeg to extract frames from a video.")

#     stream_info = analyze_stream(logger, str(input_video_path))

#     command = [
#         "ffmpeg",
#         "-y",  # (optional) overwrite output file if it exists
#         "-i",
#         str(input_video_path),
#         "-vf",
#         f"select={get_ffmpeg_select_argument(frames_sequence, setpts_filter)}",
#         "-an",  # _The presence of "-an" disables audio stream selection for_ its output
#         str(output_video_path),
#     ]

#     # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
#     # progress, use 'universal_newlines' argument
#     # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
#     ffmpeg_process = subprocess.Popen(
#         command,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.STDOUT,
#         universal_newlines=True,
#     )
#     return ffmpeg_to_tqdm(
#         logger,
#         ffmpeg_process,
#         duration=stream_info.get("duration"),
#         tqdm_desc="FFMPEG select subset of frames",
#     ).returncode


# Adapted from CLIP4Clip: https://github.com/ArrowLuo/CLIP4Clip/blob/master/preprocess/compress_video.py
# but don't change the frame rate as be which to have different frame selection strategies
# use ffmpeg_to_tqdm helper
# and directly extract a sequence of frames
def scale_compress_select_sequence(
    input_video_path: Path,
    output_video_path: Path,
    frames_sequence: List[int],
    N: int,
) -> int:
    logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
    logger.debug("Using ffmpeg to scale and compress and extract sub sequence")

    stream_info = analyze_stream(logger, str(input_video_path))

    # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
    # progress, use 'universal_newlines' argument
    # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
    ffmpeg_process = subprocess.Popen(
        f"ffmpeg -y -i {str(input_video_path)} -vf \"scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)',select={get_ffmpeg_select_argument(frames_sequence, N)}\" -map 0:v -vcodec libx264 {str(output_video_path)}",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )

    ffmpeg_to_tqdm(
        logger,
        ffmpeg_process,
        duration=stream_info.get("duration"),
        tqdm_desc="FFMPEG scale down to 224, select frames and encode libx264",
    )

    ffmpeg_process.wait()

    return ffmpeg_process.returncode

In [4]:
short_fp_df = pd.read_csv(Path("..") / "data" / "handcrafted-metadata" / "short_sequences_fp.csv", index_col="filename")
short_tp_df = pd.read_csv(Path("..") / "data" / "handcrafted-metadata" / "short_sequences_tp.csv", index_col="filename")

short_df = pd.concat([short_fp_df, short_tp_df])

short_df["StartTime"] = pd.to_timedelta(short_df["StartTime"])
short_df["EndTime"] = pd.to_timedelta(short_df["EndTime"])

In [5]:
short_df["StartFrame"] = short_df["StartTime"].dt.seconds * 25
short_df["EndFrame"] = short_df["EndTime"].dt.seconds * 25
short_df["DurationFrame"] = short_df["EndFrame"] - short_df["StartFrame"]
short_df

,video_duration,StartTime,EndTime,Distance,SubjectApproachType,SubjectDescription,SubjectOrientation,Classification,Distraction,Stage,Weather.Clouds,Weather.Fog,Weather.Rain,Weather.Snow,Weather.TimeOfDay,StartFrame,EndFrame,DurationFrame
filename,,,,,,,,,,,,,,,,,,
SZTR/video/SZTRA202b06.mov,101.00,0 days 00:00:59,0 days 00:01:34,NaN,NaN,NaN,NaN,FP,NaN,2,None,False,False,False,Day,1475,2350,875
SZTR/video/SZTRA104b10.mov,104.48,0 days 00:00:03,0 days 00:00:37,NaN,NaN,NaN,NaN,FP,NaN,1,Overcast,False,False,True,Day,75,925,850
SZTR/video/SZTRA201a09.mov,119.48,0 days 00:00:58,0 days 00:01:22,NaN,NaN,NaN,NaN,FP,NaN,2,NaN,False,False,False,Night,1450,2050,600
SZTR/video/SZTRA202b11.mov,119.60,0 days 00:01:26,0 days 00:01:44,NaN,NaN,NaN,NaN,FP,NaN,2,None,False,False,False,Day,2150,2600,450
SZTR/video/SZTRA203a12.mov,130.32,0 days 00:00:55,0 days 00:01:14,NaN,NaN,NaN,NaN,FP,NaN,2,Overcast,False,False,False,Day,1375,1850,475
SZTR/video/SZTRA103a17.mov,36.40,0 days 00:00:09,0 days 00:00:36,10.0,Walk With Ladder,One person,Perpendicular,TP,NaN,1,Overcast,False,False,False,Day,225,900,675
SZTR/video/SZTRA203a17.mov,36.40,0 days 00:00:10,0 days 00:00:36,30.0,Walk With Ladder,One person,Perpendicular,TP,NaN,2,Overcast,False,False,False,Day,250,900,650
SZTR/video/SZTRA202b13.mov,39.20,0 days 00:00:19,0 days 00:00:39,10.0,Walk With Ladder,One person,Perpendicular,TP,NaN,2,None,False,False,False,Day,475,975,500
SZTR/video/SZTRA102b13.mov,39.20,0 days 00:00:20,0 days 00:00:39,30.0,Walk With Ladder,One person,Perpendicular,TP,NaN,1,None,False,False,False,Day,500,975,475


In [6]:
short_df["OutFilename"] = (
    short_df.index.to_series().apply(
        lambda f: str(Path(f).parent / Path(f).stem).replace("/", "_")
    )
    + "_"
    + short_df["StartTime"].dt.seconds.apply(
        lambda secs: strftime("%H_%M_%S", gmtime(secs))
    )
    + "_"
    + short_df["Classification"]
    + ".mp4"
)
short_df["OutFilename"].head()

filename
SZTR/video/SZTRA202b06.mov    SZTR_video_SZTRA202b06_00_00_59_FP.mp4
SZTR/video/SZTRA104b10.mov    SZTR_video_SZTRA104b10_00_00_03_FP.mp4
SZTR/video/SZTRA201a09.mov    SZTR_video_SZTRA201a09_00_00_58_FP.mp4
SZTR/video/SZTRA202b11.mov    SZTR_video_SZTRA202b11_00_01_26_FP.mp4
SZTR/video/SZTRA203a12.mov    SZTR_video_SZTRA203a12_00_00_55_FP.mp4
Name: OutFilename, dtype: object

In [7]:
trivial_twelve = partial(trivial, 12)

for _, row in tqdm(short_df.iterrows(), desc="Extract sequences"):
    print(
        Path("..") / row.name,
        Path("..") / "data" / "sequences" / row["OutFilename"],
        (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(),
        1 / 12,
    )
    result = scale_compress_select_sequence(
        Path("..") / row.name,
        Path("..") / "data" / "sequences" / row["OutFilename"],
        (row["StartFrame"] + np.array(trivial_twelve(row["DurationFrame"]))).tolist(),
        12,
    )

    assert result == 0


Extract sequences: 0it [00:00, ?it/s]

../SZTR/video/SZTRA202b06.mov ../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 [1475, 1487, 1499, 1511, 1523, 1535, 1547, 1559, 1571, 1583, 1595, 1607, 1619, 1631, 1643, 1655, 1667, 1679, 1691, 1703, 1715, 1727, 1739, 1751, 1763, 1775, 1787, 1799, 1811, 1823, 1835, 1847, 1859, 1871, 1883, 1895, 1907, 1919, 1931, 1943, 1955, 1967, 1979, 1991, 2003, 2015, 2027, 2039, 2051, 2063, 2075, 2087, 2099, 2111, 2123, 2135, 2147, 2159, 2171, 2183, 2195, 2207, 2219, 2231, 2243, 2255, 2267, 2279, 2291, 2303, 2315, 2327, 2339, 2349] 0.08333333333333333



FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/101.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/101.0 [00:04<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/101.0 [00:04<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/101.0 [00:05<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264: 103.0segment [00:06, 16.79segment/s]            
Extract sequences: 1it [00:06,  6.26s/it]

../SZTR/video/SZTRA104b10.mov ../data/sequences/SZTR_video_SZTRA104b10_00_00_03_FP.mp4 [75, 87, 99, 111, 123, 135, 147, 159, 171, 183, 195, 207, 219, 231, 243, 255, 267, 279, 291, 303, 315, 327, 339, 351, 363, 375, 387, 399, 411, 423, 435, 447, 459, 471, 483, 495, 507, 519, 531, 543, 555, 567, 579, 591, 603, 615, 627, 639, 651, 663, 675, 687, 699, 711, 723, 735, 747, 759, 771, 783, 795, 807, 819, 831, 843, 855, 867, 879, 891, 903, 915, 924] 0.08333333333333333



FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/104.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:01<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:01<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:03<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:03<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/104.0 [00:04<?, ?segment/s]
FFMPEG scale down to

...


FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/39.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/39.0 [00:01<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/39.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264: 40.0segment [00:02, 15.16segment/s]            
Extract sequences: 8it [00:43,  4.07s/it]

../SZTR/video/SZTRA102b13.mov ../data/sequences/SZTR_video_SZTRA102b13_00_00_20_TP.mp4 [500, 512, 524, 536, 548, 560, 572, 584, 596, 608, 620, 632, 644, 656, 668, 680, 692, 704, 716, 728, 740, 752, 764, 776, 788, 800, 812, 824, 836, 848, 860, 872, 884, 896, 908, 920, 932, 944, 956, 968, 974] 0.08333333333333333



FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/39.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/39.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/39.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264: 40.0segment [00:02, 13.76segment/s]            
Extract sequences: 9it [00:46,  3.74s/it]

../SZTR/video/SZTRA203b17.mov ../data/sequences/SZTR_video_SZTRA203b17_00_00_13_TP.mp4 [325, 337, 349, 361, 373, 385, 397, 409, 421, 433, 445, 457, 469, 481, 493, 505, 517, 529, 541, 553, 565, 577, 589, 601, 613, 625, 637, 649, 661, 673, 685, 697, 709, 721, 733, 745, 757, 769, 781, 793, 805, 817, 829, 841, 853, 865, 877, 889, 901, 913, 925, 937, 949, 961, 973, 985, 997, 1009, 1021, 1033, 1045, 1049] 0.08333333333333333



FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0/44.0 [00:00<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/44.0 [00:01<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/44.0 [00:01<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264:   0%|          | 0.0/44.0 [00:02<?, ?segment/s]
FFMPEG scale down to 224, select frames and encode libx264: 46.0segment [00:02, 15.79segment/s]                    
Extract sequences: 10it [00:49,  4.99s/it]


In [27]:
def get_ffmpeg_select_argument(frames_sequence: List[int], setpts_filter: float) -> str:
    """Produce a string to be used with the FFMPEG command and its "select" argument.

    :param frames_sequence: linear or not list of frames to extract.
                            Example: 0, 12, 24, 36, 48, 60, ...
                            or:      6, 23, 27, 42, 50, 68, ...
    :param setpts_filter: from FFMPEG manual page:
                            "setpts" filter, which only sets timestamps and otherwise passes the frames unchanged
                          It is expected to be smaller than 1 in can less frames are selected and it is wished to
                          speed up the video.
                          Usually, it is expected to get 1/FRAMES_SKIP
                          Example: *0.25* (for a FRAMES_SKIP of 4)
    :return: string to be used to run the FFMPEG command for the "select" argument like: ffmpeg -i SZTRA201a08_224.mp4 -vf select='eq(n\,0)+eq(n\,12)+eq(n\,24)+eq(n\,36)+eq(n\,48)+eq(n\,60), setpts=0.08333333333*PTS' -an SZTRA201a08_224_12_random.mp4
    """
    # eq(n\\,0)+eq(n\\,12)+eq(n\\,24)+eq(n\\,36)+eq(n\\,48)
    eq_chain = "+".join([f"eq(n\,{n})" for n in frames_sequence])
    setpts = f"setpts={setpts_filter}*PTS"
    return f"'{eq_chain}',{setpts}"

def scale_compress_video(
    frames_sequence: List[int],
    setpts_filter: float,
    input_video_path: Path,
    output_video_path: Path,) -> int:
    logger = logging.getLogger(f"ffmpeg-{str(output_video_path)}")
    logger.debug("Using ffmpeg to scale and compress an original video.")

    stream_info = analyze_stream(logger, str(input_video_path))

    command = [
        "ffmpeg",
        "-y",  # (optional) overwrite output file if it exists
        "-i",
        str(input_video_path),
        "-vf",

        f"scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)'",  # scale to 224
        # f"\"scale='if(gt(a,1),trunc(oh*a/2)*2,224)':'if(gt(a,1),224,trunc(ow*a/2)*2)', select={get_ffmpeg_select_argument(frames_sequence, setpts_filter)}\"",  # scale to 224

        "-map",
        "0:v",
        "-vcodec",
        "libx264",  # additionally, change the codec for better

        # "-an",  # _The presence of "-an" disables audio stream selection for_ its output

        # compression
        # display codecs with: `ffmpeg -codecs`
        #    '-r', '3',  # frames per second
        str(output_video_path),
    ]
    # to avoid carriage return ('\r') in ffmpeg output, to mess with the reading of the
    # progress, use 'universal_newlines' argument
    # https://github.com/chriskiehl/Gooey/issues/495#issuecomment-614991802
    ffmpeg_process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )

    ffmpeg_to_tqdm(
        logger,
        ffmpeg_process,
        duration=stream_info.get("duration"),
        tqdm_desc="FFMPEG scale down to 224 and encode libx264",
    )

    ffmpeg_process.wait()

    return ffmpeg_process.returncode


root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
root.addHandler(handler)


scale_compress_video(trivial_twelve(101 * 25), 1 / 12, Path("../SZTR/video/SZTRA202b06.mov"), Path("../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4"))

2022-09-05 01:37:28,421 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - Using ffmpeg to scale and compress an original video.
2022-09-05 01:37:28,421 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - Using ffmpeg to scale and compress an original video.
2022-09-05 01:37:28,422 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - Calling PIPE child process for ffmpeg: ['ffmpeg', '-hide_banner', '-i', '../SZTR/video/SZTRA202b06.mov']
2022-09-05 01:37:28,422 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - Calling PIPE child process for ffmpeg: ['ffmpeg', '-hide_banner', '-i', '../SZTR/video/SZTRA202b06.mov']


FFMPEG scale down to 224 and encode libx264:   0%|          | 0/101.0 [00:00<?, ?segment/s]

2022-09-05 01:37:28,648 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] ffmpeg version 5.1 Copyright (c) 2000-2022 the FFmpeg developers
2022-09-05 01:37:28,648 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] ffmpeg version 5.1 Copyright (c) 2000-2022 the FFmpeg developers
2022-09-05 01:37:28,650 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
2022-09-05 01:37:28,650 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
2022-09-05 01:37:28,651 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-liba

FFMPEG scale down to 224 and encode libx264:   0%|          | 0.0/101.0 [00:00<?, ?segment/s]

2022-09-05 01:37:29,658 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame=  157 fps=0.0 q=28.0 size=       0kB time=00:00:04.12 bitrate=   0.1kbits/s speed= 8.2x
2022-09-05 01:37:29,658 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame=  157 fps=0.0 q=28.0 size=       0kB time=00:00:04.12 bitrate=   0.1kbits/s speed= 8.2x


...

2022-09-05 01:37:35,952 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame= 2425 fps=345 q=28.0 size=       0kB time=00:01:34.84 bitrate=   0.0kbits/s speed=13.5x
2022-09-05 01:37:35,952 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame= 2425 fps=345 q=28.0 size=       0kB time=00:01:34.84 bitrate=   0.0kbits/s speed=13.5x


FFMPEG scale down to 224 and encode libx264:  93%|█████████▎| 94.0/101.0 [00:07<00:00, 16.76segment/s]

2022-09-05 01:37:35,954 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame= 2525 fps=346 q=-1.0 Lsize=     260kB time=00:01:40.88 bitrate=  21.1kbits/s speed=13.8x
2022-09-05 01:37:35,954 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] frame= 2525 fps=346 q=-1.0 Lsize=     260kB time=00:01:40.88 bitrate=  21.1kbits/s speed=13.8x
2022-09-05 01:37:35,955 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] video:230kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 13.250824%
2022-09-05 01:37:35,955 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] video:230kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 13.250824%
2022-09-05 01:37:35,956 - ffmpeg-../data/sequences/SZTR_video_SZTRA202b06_00_00_59_FP.mp4 - DEBUG - [ffmpeg] [libx264 @ 0x7fc72572f240] frame I:11    Avg QP:19.28  size: 1298

FFMPEG scale down to 224 and encode libx264: 201.0segment [00:07, 27.10segment/s]                     


0

FFMPEG select subset of frames: 106.0segment [00:00, 353.38segment/s]           


0

In [22]:
trivial_twelve(101 * 25)

[600,
 612,
 624,
 636,
 648,
 660,
 672,
 684,
 696,
 708,
 720,
 732,
 744,
 756,
 768,
 780,
 792,
 804,
 816,
 828,
 840,
 852,
 864,
 876,
 888,
 900,
 912,
 924,
 936,
 948,
 960,
 972,
 984,
 996,
 1008,
 1020,
 1032,
 1044,
 1056,
 1068,
 1080,
 1092,
 1104,
 1116,
 1128,
 1140,
 1152,
 1164,
 1176,
 1188,
 1200,
 1212,
 1224,
 1236,
 1248,
 1260,
 1272,
 1284,
 1296,
 1308,
 1320,
 1332,
 1344,
 1356,
 1368,
 1380,
 1392,
 1404,
 1416,
 1428,
 1440,
 1452,
 1464,
 1476,
 1488,
 1500,
 1512,
 1524,
 1536,
 1548,
 1560,
 1572,
 1584,
 1596,
 1608,
 1620,
 1632,
 1644,
 1656,
 1668,
 1680,
 1692,
 1704,
 1716,
 1728,
 1740,
 1752,
 1764,
 1776,
 1788,
 1800,
 1812,
 1824,
 1836,
 1848,
 1860,
 1872,
 1884,
 1896,
 1908,
 1920,
 1932,
 1944,
 1956,
 1968,
 1980,
 1992,
 2004,
 2016,
 2028,
 2040,
 2052,
 2064,
 2076,
 2088,
 2100,
 2112,
 2124,
 2136,
 2148]